In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls -al drive/app
!pip install tsfresh

In [0]:
import pandas as pd
raw_data=pd.read_csv("drive/app/seagate_model_ST4000DM000_2015_2017_sorted_30day.csv")
data=pd.read_csv("drive/app/seagate_model_ST4000DM000_2015_2017_sorted_7days.csv")
data['date']=pd.to_datetime(data['date'])
data['date']=data['date'].dt.date
names=data.dtypes.index
data=data.drop([s for s in names if "normalized" in s],axis=1)
data=data.drop(['failure'],axis=1)
data=data.dropna(axis=1, how='any')
data=data.loc[:, ['serial_number','date','smart_5_raw','smart_7_raw','smart_183_raw','smart_184_raw','smart_187_raw','smart_188_raw','smart_197_raw','smart_198_raw']]
#data=data.iloc[:,0:3]
#print(data)
#data.to_csv("drive/app/relevant_columns.csv")
#impute(data)


In [0]:
import numpy as np
failure_data=pd.read_csv("drive/app/failure_data.csv")
#failure_data = pd.Series(failure_data['failure'].values, index=failure_data['serial_number'])
failure_data=np.array([i for i in failure_data['failure']])
#print(failure_data)

In [0]:
from tsfresh import extract_features
extracted_features = extract_features(data, column_id="serial_number", column_sort="date")
impute(extracted_features)
extracted_features.to_csv("drive/app/all_extarcted_features.csv")

In [0]:
from tsfresh import select_features
selected_features=select_features(extracted_features,failure_data,fdr_level=0.05)
selected_features.to_csv("drive/app/selected_features.csv")

In [0]:
from tsfresh import extract_relevant_features
features_filtered_direct = extract_relevant_features(data, failure_data,column_id="serial_number", column_sort="date", fdr_level=0.05)
features_filtered_direct.to_csv("drive/app/features_filtered_direct_7days.csv")
#features_filtered_direct=pd.read_csv("drive/app/features_filtered_direct.csv")
#print(features_filtered_direct)

Feature Extraction:  20%|██        | 1/5 [06:00<24:00, 360.05s/it]

In [11]:
#from tsfresh.feature_selection.relevance import calculate_relevance_table 
#p_value_matrix=calculate_relevance_table(features_filtered_direct,failure_data, fdr_level=0.05)
print(features_filtered_direct.shape)

(2833, 853)
